# Importing Required Modules

In [1]:
import cv2
import numpy as np
import requests
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split


# Downloading the HAAR Data File

In [2]:
def gethaar(url="https://raw.githubusercontent.com/opencv/opencv/22f0bcaf8f4fb406ae251830cf99b7ea05de40e4/data/haarcascades/haarcascade_frontalface_default.xml"):    
    haar = requests.get(url)
    return haar

# Saving the HAAR Data File

In [3]:
def savehaar(haar):
    datafile = open("data.xml", "wb")
    datafile.write(haar.content)
    datafile.close()

# Using the Classifier to get the Face Cooridinate

In [4]:
def classify():
    haar_data = cv2.CascadeClassifier('data.xml')
    return haar_data

# Using the Default Video Capture Device

In [5]:
def setcam():
    capture = cv2.VideoCapture(0)
    return capture
def releasecam(capture):
    capture.release()

# Storing the Mask And Umasked Data / Predict

In [65]:
haar_data = classify()
capture = setcam()
flag, img = capture.read()
faces = haar_data.detectMultiScale(img)
for x, y, w, h in faces:
     face = img[y:y+h, x:x+w, :]
     face = cv2.resize(face, (50, 50))
face = face.reshape(3, -1)
face = fit(face)
face

array([[-1.24819880e+03, -1.93573827e+03,  1.45080204e-12],
       [-1.20498812e+03,  1.95860338e+03,  1.45080204e-12],
       [ 2.45318692e+03, -2.28651077e+01,  1.45080204e-12]])

In [7]:
def facedata(haar_data, data=[], predicting=False):
    capture = setcam()
    font = cv2.FONT_HERSHEY_COMPLEX
    names = {0: 'Mask', 1: 'No Mask'}
    prepared = False
    while True:
        flag, img = capture.read()
        if flag:
            faces = haar_data.detectMultiScale(img)
            for x, y, w, h in faces:
                if not predicting:
                    cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,255), 4)
                face = img[y:y+h, x:x+w, :]
                face = cv2.resize(face, (50, 50))
                if predicting:
                    if not prepared:
                        load()
                        reshape()
                        X = combine(with_mask, without_mask)
                        labels = divide(X)
                        X = fit(X)
                        svm = train(X, labels)
                        prepared = True
                    face = face.reshape(3, -1)
                    face = fit(face)
                    pred = svm.predict(face)[0]
                    n = names[int(pred)]
                    
                    if n == 'Mask':
                        cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 4)
                    else:
                        cv2.rectangle(img, (x,y), (x+w, y+h), (0,0,255), 4)
                    cv2.putText(img, n, (x, y), font, 1, (244, 250, 250), 2)
                if len(data)<250 and not predicting:
                    data.append(face)
            cv2.imshow("Seal's Face Mask Detector", img)
            if cv2.waitKey(10) == 27 or len(data)>=250:
                break
    releasecam(capture)
    cv2.destroyAllWindows()
    if not predicting:
        return data


# Storing Unmasked Data

In [8]:
def unmasked(haar_data):
    print("Don't Wear A Mask. Press Escape to Exit.")
    data = facedata(haar_data, data=[])
    np.save('without_mask.npy', data)
    return True

# Storing Masked Data

In [9]:
def masked(haar_data):
    print("Wear A Mask. Press Escape to Exit.")
    data = facedata(haar_data, data=[])
    np.save('with_mask.npy', data)
    return True

# Loading the saved files

In [10]:
def load():
    global with_mask, without_mask
    with_mask = np.load("with_mask.npy")
    without_mask = np.load("without_mask.npy")

# Reshaping the saved files

In [11]:
def reshape():
    global with_mask, without_mask
    with_mask = with_mask.reshape(250, 50*50*3)
    without_mask = without_mask.reshape(250, 50*50*3)

# Combining the Reshaped Files

In [12]:
def combine(lst1, lst2):
    X = np.r_[lst1, lst2]
    return X

<function __main__.combine(lst1, lst2)>

# Segregating the Dependent Variable 

In [13]:
def divide(X):
    labels = np.zeros(X.shape[0])
    labels[250:] = 1.0
    return labels

# Fitting Data

In [37]:
def fit(X, n=3):
    pca = PCA(n_components=n)
    X = pca.fit_transform(X)
    return X

# Training

In [15]:
def train(x_train, y_train):
    global svm
    svm = SVC()
    x_train, x_useless, y_train, y_useless = train_test_split(x_train, y_train, test_size=0.01)
    svm.fit(x_train, y_train)
    return svm

# Predicting

In [16]:
def predict(var, svm):
    y_pred = svm.predict(var)
    return y_pred

# Calculating Accuracy

In [17]:
def accuracy(y_train, y_pred):
    return accuracy_score(y_train, y_pred)


# Prepare Predictor

In [56]:
def preparepredictor():
    load()
    reshape()
    X = combine(with_mask, without_mask)
    print(X)
    labels = divide(X)
    X = fit(X)
    svm = train(X, labels)
    print(X)
# preparepredictor()

[[255   0 255 ... 238   3 238]
 [255   0 255 ... 236   4 236]
 [255   0 255 ... 244   4 244]
 ...
 [255   0 255 ... 241   6 241]
 [255   0 255 ... 241   4 241]
 [255   0 255 ... 241   5 241]]
[[-705.02377516   44.40617662   -7.33217215]
 [-746.76770613  113.78071253  -35.78541067]
 [-885.25798477  105.81220561  285.16161543]
 ...
 [ 147.45715737 -502.37729404 -129.73319405]
 [  34.0106082  -582.39868143 -117.41338564]
 [ 112.10819214 -507.55584531  -52.84099799]]


# Driver Function

In [19]:
def resetconfirmation():
    return "n"
    # y_pred = predict(X, svm)
    # confidence = accuracy(labels, y_pred)
    # print(y_pred, confidence)
def main(ispredicting  = False):
    if not ispredicting:
        url = "https://raw.githubusercontent.com/opencv/opencv/22f0bcaf8f4fb406ae251830cf99b7ea05de40e4/data/haarcascades/haarcascade_frontalface_default.xml"
        haar = gethaar(url)
        savehaar(haar)
        haar_data = classify()
        confirm = resetconfirmation()
        while confirm.lower() != "y":
            print("Remove a Mask if you are wearing one.\nSit In a well-lit environment.\nPress Y to confirm: ")
            confirm = input()
        unmasked(haar_data)
        confirm = resetconfirmation()
        while confirm.lower() != "y":
            print("Wear a Mask if you are not wearing one.\nSit In a well-lit environment.\nPress Y to confirm: ")
            confirm = input()
        masked(haar_data)
    else:
        haar_data = classify()
        facedata(haar_data, [], predicting=ispredicting)


# Calling Main

In [20]:
if __name__ == "__main__":
    for i in range(2):
        main(ispredicting=(False if i%2==0 else True))

Remove a Mask if you are wearing one.
Sit In a well-lit environment.
Press Y to confirm: 


KeyboardInterrupt: Interrupted by user